<a href="https://colab.research.google.com/github/StefanoGalli99/EUOptionFlow/blob/main/OptionFlow_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
get_ipython().magic('reset -sf')

In [ ]:
!pip install requests beautifulsoup4 pandas openpyxl
import os
import pandas as pd
import sqlite3
from pathlib import Path
from urllib.parse import urljoin
import matplotlib.pyplot as plt
import requests # import the requests library
import time
from bs4 import BeautifulSoup  # import BeautifulSoup

In [ ]:
def download_excel_files():
    # URL del sito web da cui scaricare i file Excel
    base_url = "https://live.euronext.com"
    url = urljoin(base_url, "/en/resources/statistics/nextday-derivatives")

    try:
        # Richiedi all'utente il numero di file da scaricare
        while True:
            num_files = input("Quanti file Excel vuoi scaricare? (Inserisci un numero o 'tutti' per scaricarli tutti): ")
            if num_files.lower() == 'tutti':
                num_files = float('inf')  # Imposta un numero infinito per scaricare tutti i file
                break
            try:
                num_files = int(num_files)
                if num_files <= 0:
                    print("Per favore inserisci un numero positivo.")
                else:
                    break
            except ValueError:
                print("Per favore inserisci un numero valido o 'tutti'.")

        # Effettua una richiesta GET alla pagina web con timeout
        print("\nConnessione al sito web...")
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        # Analizza il contenuto della pagina con BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')

        # Filtra solo i link Excel
        excel_links = [link for link in links if link.get('href') and link.get('href').endswith('.xlsx')]

        if not excel_links:
            print("Nessun file Excel trovato nella pagina.")
            return

        print(f"\nTrovati {len(excel_links)} file Excel disponibili.")
        if num_files != float('inf'):
            print(f"Verranno scaricati i primi {min(num_files, len(excel_links))} file.")

        # Cartella di destinazione per i file scaricati
        folder_path = "excel_files"
        os.makedirs(folder_path, exist_ok=True)

        files_downloaded = 0
        files_failed = 0

        for i, link in enumerate(excel_links):
            if files_downloaded >= num_files:
                break

            href = link.get('href')
            excel_link = urljoin(base_url, href) if not href.startswith('http') else href
            file_name = os.path.basename(href)
            file_path = os.path.join(folder_path, file_name)

            # Controllo se il file esiste già
            if os.path.exists(file_path):
                print(f"\nIl file '{file_name}' esiste già. Salto il download.")
                continue

            try:
                time.sleep(1)
                print(f"\nDownloading {file_name} ({files_downloaded + 1}/{min(num_files, len(excel_links))})...")
                excel_file = requests.get(excel_link, timeout=30)
                excel_file.raise_for_status()

                # Verifica che il contenuto sia un file Excel
                content_type = excel_file.headers.get('content-type', '')
                if 'application/vnd.openxmlformats-officedocument.spreadsheetml' not in content_type:
                    print(f"Attenzione: Il file {file_name} potrebbe non essere un file Excel valido.")

                with open(file_path, "wb") as f:
                    f.write(excel_file.content)

                files_downloaded += 1
                print(f"File '{file_name}' scaricato con successo.")

            except requests.RequestException as e:
                files_failed += 1
                print(f"Errore nel download del file {file_name}: {e}")

        # Report finale
        print("\nRiepilogo download:")
        print(f"File scaricati con successo: {files_downloaded}")
        print(f"File non scaricati a causa di errori: {files_failed}")
        print(f"I file sono stati salvati in: {os.path.abspath(folder_path)}")

    except requests.RequestException as e:
        print(f"Errore nell'accesso alla pagina web: {e}")
    except Exception as e:
        print(f"Errore imprevisto: {e}")

if __name__ == "__main__":
    download_excel_files()


Quanti file Excel vuoi scaricare? (Inserisci un numero o 'tutti' per scaricarli tutti): 5

Connessione al sito web...

Trovati 5538 file Excel disponibili.
Verranno scaricati i primi 5 file.

File 'Derivatives%2020241024.xlsx' scaricato con successo.

Il file 'Derivatives%2020241024.xlsx' esiste già. Salto il download.

File 'Derivatives%2020241023.xlsx' scaricato con successo.

Il file 'Derivatives%2020241023.xlsx' esiste già. Salto il download.

File 'Derivatives%2020241022.xlsx' scaricato con successo.

Il file 'Derivatives%2020241022.xlsx' esiste già. Salto il download.

File 'Derivatives%2020241021.xlsx' scaricato con successo.

Il file 'Derivatives%2020241021.xlsx' esiste già. Salto il download.

File 'Derivatives%2020241018.xlsx' scaricato con successo.

Riepilogo download:
File scaricati con successo: 5
File non scaricati a causa di errori: 0
I file sono stati salvati in: /content/excel_files


In [ ]:
# Inizializzazione delle variabili
conn = None

In [ ]:
# Creazione del database e tabella
conn = sqlite3.connect('financial_data.db')
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS companies (id INTEGER PRIMARY KEY, name TEXT UNIQUE)")

# Set per tenere traccia delle aziende e della blacklist
processed_companies = set()
blacklist = set()

# Percorso della cartella
folder_path = 'excel_files'
# Elenco dei file Excel nella cartella
excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(f"Trovati {len(excel_files)} file Excel nella cartella '{folder_path}'.")

# Limita a 2 file per test
for file_name in excel_files[:2]:
    print(f"Processing file: {file_name}")  # Messaggio di debug
    file_path = os.path.join(folder_path, file_name)

    # Leggi il terzo foglio del file Excel
    df = pd.read_excel(file_path, sheet_name=2, header=None, skiprows=4)  # Salta le prime 4 righe

    for row_idx, row in df.iterrows():
        potential_name = row[5]  # La sesta colonna è in posizione 5

        # Controlla se il nome è NaN
        if pd.isna(potential_name):
            print("IL NOME DI QUESTA RIGA è NULLO")
            continue  # Se il nome è NaN, passa alla prossima iterazione

        base_name = potential_name.split('-')[0].strip()  # Estrarre la parte prima del trattino

        # Controlla se il derivato è un future, una dividend option o una european option
        if "future" in potential_name.lower() or "dividend" in potential_name.lower() or "european" in potential_name.lower():
            continue  # Salta se contiene "future", "dividend" o "european"

        # Controlla se il nome è già stato processato o è nella blacklist
        if base_name in processed_companies or base_name in blacklist:
            continue

        # Richiede input per confermare il nome
        response = input(
            f"File: '{file_name}', Riga: {row_idx + 5}, Nome azienda (completo): '{potential_name}': "
            "Questa cella contiene il nome di un'azienda? (s/no): "
        ).strip().lower()

        if response == 's':
            # Aggiungi solo la parte prima del trattino
            cursor.execute("INSERT OR IGNORE INTO companies (name) VALUES (?)", (base_name,))
            processed_companies.add(base_name)
            print(f"Azienda aggiunta: {base_name}")  # Messaggio di conferma
        else:
            # Aggiungi alla blacklist se non è valido
            blacklist.add(base_name)
            print(f"Azienda aggiunta alla blacklist: {potential_name}")

# Salva le modifiche e chiudi la connessione
conn.commit()
conn.close()
print("Tabella delle aziende popolata con successo.")


Trovati 5 file Excel nella cartella 'excel_files'.
Processing file: Derivatives%2020241022.xlsx
File: 'Derivatives%2020241022.xlsx', Riga: 25, Nome azienda (completo): 'AEX Daily Option - 22nd': Questa cella contiene il nome di un'azienda? (s/no): s
Azienda aggiunta: AEX Daily Option
File: 'Derivatives%2020241022.xlsx', Riga: 235, Nome azienda (completo): 'AEX Index - Mini Option': Questa cella contiene il nome di un'azienda? (s/no): s
Azienda aggiunta: AEX Index
File: 'Derivatives%2020241022.xlsx', Riga: 427, Nome azienda (completo): 'AEX Index Option': Questa cella contiene il nome di un'azienda? (s/no): s
Azienda aggiunta: AEX Index Option
File: 'Derivatives%2020241022.xlsx', Riga: 957, Nome azienda (completo): 'AEX Weekly - 1st Friday': Questa cella contiene il nome di un'azienda? (s/no): s
Azienda aggiunta: AEX Weekly
File: 'Derivatives%2020241022.xlsx', Riga: 1281, Nome azienda (completo): 'PSI20 Index Option': Questa cella contiene il nome di un'azienda? (s/no): s
Azienda aggiun

In [ ]:
print(processed_companies)

{'Porsche pref.', 'Worldline', 'Euronext', 'Deutsche Lufthansa', 'Verallia', 'EUE Stock Option (Underlying: iShares Core EURO STOXX 50 UCI', 'Aker BP', 'Publicis Groupe (100)', 'Symrise', 'Zalando', 'AMG N.V.', 'Allianz', 'Deutsche Boerse', 'Corn / Mais', 'AGFA GEVAERT', 'AEX Weekly', 'Brenntag', 'EDP Renovaveis', 'Salvatore Ferragamo', 'Snam', 'Siemens Healthineers', 'Covivio', 'ENAV', 'Amplifon', 'Societe Generale (100)', 'CAREL Industries', 'Azelis Group NV', 'Corbion NV', 'Norsk Hydro', 'SPIE', 'RWE AG', 'BNP Paribas (100)', 'BPER Banca', 'Pernod Ricard (100)', 'Syensqo', 'BFF Bank', 'Generali', 'Elior Group', 'Orange SA (100)', 'Alfen', 'Sanofi ex', 'Hera', 'Ayvens', 'Intesa Sanpaolo', 'Saint Gobain (100)', 'Ferrovial', 'Commerzbank', 'Atos (100)', 'MFE B', 'Rai Way', 'Sanlorenzo', 'GTT', 'Ubisoft Entertainment', 'Webuild', 'AB Science', 'Kerry Group', 'Legrand SA  Std Options', 'Safran SA (100)', 'Axa', 'ASR Nederland', 'RELX (EUR)', 'Rubis', 'Equinor', 'TGS ASA', 'GEOX', 'Acea',

In [ ]:
# Creazione del database e delle tabelle
conn = sqlite3.connect('financial_data.db')
cursor = conn.cursor()

# Tabella delle aziende
cursor.execute("CREATE TABLE IF NOT EXISTS companies (id INTEGER PRIMARY KEY, name TEXT UNIQUE)")
# Tabella delle date per ogni azienda
cursor.execute("CREATE TABLE IF NOT EXISTS company_dates (company_id INTEGER, date TEXT, UNIQUE(company_id, date))")

# Carica aziende già esistenti
cursor.execute("SELECT name, id FROM companies")
company_ids = {name: company_id for name, company_id in cursor.fetchall()}

# Percorso della cartella
folder_path = 'excel_files'
# Elenco dei file Excel nella cartella (limita a 2 file per test)
excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')][:2]

for file_name in excel_files:
    print(f"Processing file: {file_name}")
    file_path = os.path.join(folder_path, file_name)

    # Leggi il terzo foglio del file Excel
    df = pd.read_excel(file_path, sheet_name=2, header=None, skiprows=4)  # Salta le prime 4 righe

    # Controlla ogni riga per ogni azienda
    for row_idx, row in df.iterrows():
        potential_name = row[5]  # La sesta colonna è in posizione 5
        if pd.isna(potential_name):
            continue

        base_name = potential_name.split('-')[0].strip()
        # Escludi tipi di contratti indesiderati
        if any(word in potential_name.lower() for word in ["future", "dividend", "european"]):
            continue

        # Aggiungi la data se l'azienda è nel database
        date_value = row[0]  # Assumendo che la data sia nella prima colonna
        if pd.notna(date_value) and base_name in company_ids:
            company_id = company_ids[base_name]

            # Verifica e conversione di date_value a stringa
            date_value_str = str(date_value) if not isinstance(date_value, str) else date_value

            # Inserisci la data per l'azienda, se non già esistente
            cursor.execute("INSERT OR IGNORE INTO company_dates (company_id, date) VALUES (?, ?)", (company_id, date_value_str))
            print(f"Aggiunta data per {base_name}: {date_value_str}")

# Salva le modifiche e chiudi la connessione
conn.commit()
conn.close()
print("Date per azienda aggiunte con successo.")


Output streaming troncato alle ultime 5000 righe.
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per Societe Generale (100): 2024-10-22 00:00:00
Aggiunta data per So

KeyboardInterrupt: 

In [ ]:
def ensure_database_exists():
    """
    Ensure database and tables exist, return True if successful, False otherwise.
    """
    try:
        conn = sqlite3.connect('options_data.db')
        cursor = conn.cursor()

        # Create tables if they don't exist
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS options_data (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                company TEXT,
                contract_type TEXT,
                strike_price REAL,
                expiry_date TEXT,
                open_interest INTEGER,
                volume INTEGER,
                bid REAL,
                ask REAL
            )
        ''')

        # Create indices
        cursor.execute('''
            CREATE INDEX IF NOT EXISTS idx_options_lookup
            ON options_data(company, date)
        ''')

        cursor.execute('''
            CREATE INDEX IF NOT EXISTS idx_strike_price
            ON options_data(strike_price)
        ''')

        conn.commit()

        # Verify table exists
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='options_data'")
        if not cursor.fetchone():
            print("Failed to create options_data table")
            return False

        # Verify if table has data
        cursor.execute("SELECT COUNT(*) FROM options_data")
        count = cursor.fetchone()[0]

        if count == 0:
            print("Database exists but is empty. Need to process Excel files.")
            return False

        print(f"Database exists and contains {count} records")
        return True

    except Exception as e:
        print(f"Error checking database: {e}")
        return False
    finally:
        if conn:
            conn.close()

import re



In [ ]:
def clean_column_labels(df):
    """
    Rimuove formattazioni e caratteri speciali dalle etichette di colonna.
    """
    cleaned_columns = {}
    for col in df.columns:
        clean_label = re.sub(r'[\s\n]+', ' ', col).strip().lower()  # Rimuove spazi multipli e a capo
        cleaned_columns[col] = clean_label
    df.rename(columns=cleaned_columns, inplace=True)

    # Mappatura delle colonne in base alle etichette standard
    column_map = {
        'date': 'date',
        'market place': 'market_place',
        'product group': 'product_group',
        'contract type': 'contract_type',
        'contract name': 'contract_name',
        'contract code': 'contract_code',
        'last trading date': 'last_trading_date',
        'expiry month': 'expiry_month',
        'exercise price': 'exercise_price',
        'instrument type': 'instrument_type',
        'valuation coefficient': 'valuation_coefficient',
        'exercise type': 'exercise_type',
        'amr': 'amr',
        'isin': 'isin',
        'currency': 'currency',
        'first price': 'first_price',
        'highest price': 'highest_price',
        'lowest price': 'lowest_price',
        'last price': 'last_price',
        'settlement price': 'settlement_price',
        'volume (total)': 'volume_total',
        'volume (wholesale)': 'volume_wholesale',
        'number of trades (total)': 'num_trades_total',
        'value of volume (total, euro)': 'value_volume_total_euro',
        'premium turnover (total, euro)': 'premium_turnover_total_euro',
        'open interest': 'open_interest',
        'settlement volatility': 'settlement_volatility',
        'settlement delta': 'settlement_delta'
    }

    # Rinomina le colonne del dataframe in base alla mappatura
    df.rename(columns=column_map, inplace=True)
    return df



In [ ]:
def process_excel_files():
    """
    Processa i file Excel, applica la pulizia delle etichette e salva nel database.
    """
    try:
        conn = sqlite3.connect('options_data.db')
        cursor = conn.cursor()

        excel_folder = "excel_files"
        excel_files = list(Path(excel_folder).glob("*.xlsx"))

        for excel_file in excel_files:
            df = pd.read_excel(excel_file, header=3)
            df = clean_column_labels(df)  # Pulizia delle etichette

            # Filtra solo le opzioni
            df_options = df[df['contract_type'].str.contains('Options', case=False, na=False)]

            # Inserisce i dati puliti nel database
            for _, row in df_options.iterrows():
                data = (
                    row['date'], row['contract_name'].split('-')[0].strip(),
                    row['contract_type'], row['exercise_price'], row['expiry_month'],
                    row['open_interest'], row['volume_total'], row.get('bid'), row.get('ask')
                )

                cursor.execute('''
                    INSERT INTO options_data
                    (date, company, contract_type, strike_price, expiry_date, open_interest, volume, bid, ask)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', data)
            conn.commit()

    finally:
        conn.close()


In [ ]:
def plot_options_open_interest(company_name, date):
    """Create a detailed visualization of options open interest."""
    try:
        conn = sqlite3.connect('options_data.db')

        # Verify data exists for the company and date
        verify_query = '''
            SELECT COUNT(*) FROM options_data
            WHERE company = ? AND date = ?
        '''
        count = conn.execute(verify_query, (company_name, date)).fetchone()[0]

        if count == 0:
            print(f"No data found for {company_name} on {date}")
            return

        print(f"Found {count} records for {company_name} on {date}")

        # Get options data
        query = '''
            SELECT strike_price, contract_type, open_interest, expiry_date
            FROM options_data
            WHERE company = ? AND date = ?
            ORDER BY strike_price, expiry_date
        '''

        df = pd.read_sql_query(query, conn, params=(company_name, date))

        # Create visualization
        plt.figure(figsize=(15, 10))

        # Split data into calls and puts
        calls = df[df['contract_type'].str.contains('CALL', case=False, na=False)]
        puts = df[df['contract_type'].str.contains('PUT', case=False, na=False)]

        # Plot calls above x-axis
        if not calls.empty:
            plt.bar(calls['strike_price'], calls['open_interest'],
                   color='green', alpha=0.6, label='Calls')

        # Plot puts below x-axis
        if not puts.empty:
            plt.bar(puts['strike_price'], -puts['open_interest'],
                   color='red', alpha=0.6, label='Puts')

        plt.title(f'Options Open Interest Distribution\n{company_name} - {date}',
                 fontsize=14, pad=20)
        plt.xlabel('Strike Price', fontsize=12)
        plt.ylabel('Open Interest\n(Puts below axis, Calls above)', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.legend(loc='upper right')

        # Add statistics
        total_calls_oi = calls['open_interest'].sum() if not calls.empty else 0
        total_puts_oi = puts['open_interest'].sum() if not puts.empty else 0
        put_call_ratio = total_puts_oi / total_calls_oi if total_calls_oi > 0 else float('inf')

        stats_text = (f'Total Calls OI: {total_calls_oi:,.0f}\n'
                     f'Total Puts OI: {total_puts_oi:,.0f}\n'
                     f'Put/Call Ratio: {put_call_ratio:.2f}')

        plt.figtext(0.02, 0.98, stats_text,
                   bbox=dict(facecolor='white', alpha=0.8),
                   fontsize=10, va='top')

        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error creating visualization: {e}")
    finally:
        if conn:
            conn.close()



In [ ]:
# Main execution
if __name__ == "__main__":
    try:
        # Check if database exists and has data
        if not ensure_database_exists():
            print("Need to process existing Excel files...")
            if process_excel_files():
                print("Dati elaborati correttamente.")
            else:
                raise Exception("Errore nell'elaborazione dei file Excel.")

        # Get user input
        company_name = input("Enter company name: ")
        date = input("Enter date (YYYYMMDD format): ")

        # Create visualization
        plot_options_open_interest(company_name, date)

    except Exception as e:
        print(f"Error in main execution: {e}")


Database exists but is empty. Need to process Excel files.

Connecting to website...

File 'Derivatives%2020241024.xlsx' already exists. Skipping.

File 'Derivatives%2020241024.xlsx' already exists. Skipping.

File 'Derivatives%2020241023.xlsx' already exists. Skipping.

File 'Derivatives%2020241023.xlsx' already exists. Skipping.

File 'Derivatives%2020241022.xlsx' already exists. Skipping.

File 'Derivatives%2020241022.xlsx' already exists. Skipping.

File 'Derivatives%2020241021.xlsx' already exists. Skipping.

File 'Derivatives%2020241021.xlsx' already exists. Skipping.

File 'Derivatives%2020241018.xlsx' already exists. Skipping.

File 'Derivatives%2020241018.xlsx' already exists. Skipping.

File 'Derivatives%2020241017.xlsx' already exists. Skipping.

File 'Derivatives%2020241017.xlsx' already exists. Skipping.

File 'Derivatives%2020241016.xlsx' already exists. Skipping.

File 'Derivatives%2020241016.xlsx' already exists. Skipping.

File 'Derivatives%2020241015.xlsx' already exi

KeyboardInterrupt: 